In [2]:
import gym 
from gym import spaces
import numpy as np
import pygame
from pygame.locals import *
from run import GameController
from constants import *
from pacman import Pacman
from ghost import Ghosts
from nodes import NodeGroup
from pellets import PelletGroup
from fruits import Fruits
from stable_baselines3 import DQN
from stable_baselines3 import PPO
import os
import time
class PacmanEnv(gym.Env):
    metadata = {"render.modes": ["human"]}

class PacmanEnv(gym.Env):
    def __init__(self, render_mode=False):
        super(PacmanEnv, self).__init__()
        self.render_mode = render_mode

        if not render_mode:
            os.environ["SDL_VIDEODRIVER"] = "dummy"
        else:
            os.environ.pop("SDL_VIDEODRIVER", None)

        pygame.quit() 
        pygame.init()

        self.game = GameController(render_mode=render_mode)

        self.action_space = spaces.Discrete(5, start=-2)
        self.observation_space = spaces.Box(
            low=0, high=255, shape=(SCREENHEIGHT, SCREENWIDTH, 3), dtype=np.uint8
        )

    def reset(self):
        self.game.startGame(3)
        state = self.get_observation()
        return state
    
    def step(self, action):
        action = int(action)
        action = np.clip(action, 0, 4)
    
        action = action - 2
        if self.game.pacman.validDirection(action):
            self.game.pacman.direction = action 

        pelletBefore = self.game.pellets.numEaten 
        lifesBefore = self.game.pacman.life_amount   
        self.game.update()

        if self.game.pacman.target is not None and self.game.pacman.overshotTarget():
            self.game.pacman.node = self.game.pacman.target
            self.game.pacman.setPosition()

        self.game.update() 

        state = self.get_observation() 

        reward = 0

        pellet = self.game.pellets.numEaten - pelletBefore
        if pellet == 1:
            reward += 20

        fruit = None
        if self.game.fruits is not None:
            fruit = self.game.pacman.eatFruits(self.game.fruits)
            if fruit:
                reward += 20

        lifes = self.game.pacman.life_amount - lifesBefore
        if lifes == -1:
            reward -= 50


        if pellet == 0 and fruit is None:
            reward -= 2

        done = self.check_game_over()

        info = {}

        return state, reward, done, info
    
    def render(self, mode="human"):
        if self.render_mode and mode == "human":
            self.game.render()

    def get_observation(self):
        return pygame.surfarray.array3d(self.game.screen)

    def _init_pygame(self):
        if not pygame.get_init():
            pygame.init()

    def close(self):
        pygame.quit()

    def check_game_over(self):
        return self.game.pacman.life_amount == 0
    
    def change_resolution(self, width, height):
        global SCREENWIDTH, SCREENHEIGHT

        constants_path = os.path.join(os.path.dirname(__file__), "constants.py")
        with open(constants_path, "r") as file:
            lines = file.readlines()

        with open(constants_path, "w") as file:
            for line in lines:
                if line.startswith("SCREENWIDTH"):
                    file.write(f"SCREENWIDTH = {width}\n")
                elif line.startswith("SCREENHEIGHT"):
                    file.write(f"SCREENHEIGHT = {height}\n")
                else:
                    file.write(line)

        SCREENWIDTH, SCREENHEIGHT = width, height
        
        self.game.screen = pygame.display.set_mode((SCREENWIDTH, SCREENHEIGHT))
        self.game.width, self.game.height = SCREENWIDTH, SCREENHEIGHT

    def get_observation(self):
        observation = pygame.surfarray.array3d(self.game.screen)
        return np.transpose(observation, (1, 0, 2))








In [3]:
env = PacmanEnv(render_mode=False)
model = DQN("MlpPolicy", env, verbose=1, buffer_size=1000)

print("Training the model...")
for i in range(10):
    model.learn(total_timesteps=2000)
    env.reset
    print(i)

model.save("pacman_10x2000dqn_model")
env.close()

model = DQN("MlpPolicy", env, verbose=0, buffer_size=1000)

print("Training the model...")
for i in range(20):
    model.learn(total_timesteps=1000)
    env.reset
    print(i)

model.save("pacman_20x1000dqn_model")
env.close()



Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


c:\Users\jacek\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Training the model...


TypeError: Ghosts.update_ghosts() missing 1 required positional argument: 'pacman'

In [22]:
print("Switching to testing mode...")
env = PacmanEnv(render_mode=False)
state = env.reset()
rewardMain = 0
model = DQN("MlpPolicy", env, verbose=1, buffer_size=1000)

model.load("pacman_10000_dqn_model.zip")

state = env.reset()

rewards = []
durations = []
times=[]
episodes = 20
max_episode_time = 10 * 60

for episode in range(episodes):
    state = env.reset()
    episode_reward = 0
    steps = 0
    steps_time = 0
    done = False
    episode_start_time = time.time()
    print("starting episode" + str(episode))
    while not done:
        if time.time() - episode_start_time >= max_episode_time:
            print(f"Episode {episode} exceeded 10 minutes, stopping early.")
            break
        steptime = time.time()
        action, _ = model.predict(state)
        state, reward, done, _ = env.step(action)
        episode_reward += reward
        steps_time += time.time() - steptime
        steps += 1
        env.render()
    times.append(steps_time/steps)
    rewards.append(episode_reward)
    durations.append(steps)
env.close()

file = open("tests.txt", "a")
file.write("DQN model for " + str(episodes) + " episodes\n")    
for i in range(len(rewards)):
    file.write("episode" + str(i)+ " :" + "reward " + str(rewards[i]) + " duration: " + str(durations[i]) + " mean step time: " + str(times[i])+"\n")     
file.close()



Switching to testing mode...
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
starting episode0
Episode 0 exceeded 10 minutes, stopping early.
starting episode1
Episode 1 exceeded 10 minutes, stopping early.
starting episode2
Episode 2 exceeded 10 minutes, stopping early.
starting episode3
starting episode4
starting episode5
starting episode6
starting episode7
starting episode8
Episode 8 exceeded 10 minutes, stopping early.
starting episode9
Episode 9 exceeded 10 minutes, stopping early.
starting episode10
starting episode11
Episode 11 exceeded 10 minutes, stopping early.
starting episode12
starting episode13
Episode 13 exceeded 10 minutes, stopping early.
starting episode14
Episode 14 exceeded 10 minutes, stopping early.
starting episode15
Episode 15 exceeded 10 minutes, stopping early.
starting episode16
starting episode17
starting episode18
Episode 18 exceeded 10 minutes, stopping early.
starting e

In [1]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

env = PacmanEnv(render_mode=False)
    
model = PPO("CnnPolicy", env, verbose=1, n_steps=256, batch_size=64, ent_coef=0.01)

print("Training the PPO model...")
model.learn(total_timesteps=100)
model.save("pacman_ppo_model")
env.close() 

NameError: name 'PacmanEnv' is not defined

In [ ]:
print("Switching to testing mode...")
env = PacmanEnv(render_mode=True)
state = env.reset()
rewardMain = 0
model = PPO("CnnPolicy", env, verbose=1, n_steps=256, batch_size=64, ent_coef=0.01)

model.load("pacman_ppo_model.zip")

state = env.reset()

rewards = []
durations = []
episodes = 15

for episode in range(episodes):
    state = env.reset()
    episode_reward = 0
    steps = 0
    done = False

    while not done:
        action, _ = model.predict(state)
        state, reward, done, _ = env.step(action)
        episode_reward += reward
        steps += 1

    rewards.append(episode_reward)
    durations.append(steps)

file = open("tests.txt")
file.write("ppo model for {episodes} episodes")    
for i in range(rewards.count()):
    file.write("episode {i}:", "reward ",rewards[i], " duration: ", durations[i])    
    
env.close()

In [ ]:
print("Trained PPO model:")
state = env.reset()
rewardMain = 0
for _ in range(1000):
    action, _states = model.predict(state)

    state, reward, done, info = env.step(action)

    env.render()
    rewardMain += reward

    if done:
        print("Game Over")
        break

print(f"Total reward during testing: {rewardMain}")
env.close()